In [1]:
import findspark
findspark.init()
import pyspark

In [2]:
from pyspark import SparkContext
from pyspark import SparkConf

In [3]:
conf=SparkConf().set("spark.default.parallelism", 4)
sc = SparkContext(conf=conf)

In [94]:
data = sc.textFile('data_fianl_project.txt')

In [95]:
def mapper1(line):
    wordlist = line.split("\t")
    key = wordlist[1]
    value = (wordlist[0], float(wordlist[2]))
    maplist = []
    maplist.append((key, [value]))
    return maplist

In [96]:
data = data.flatMap(mapper1)

In [97]:
def reducer1(x,y):
    return x+y

#### list out the users who rate the movie.

#### The data structure is (movie, [(user_i,rate),...]).

In [98]:
movielist = data.reduceByKey(reducer1).collect()

In [99]:
movie_num = len(movielist)

In [100]:
def getpair(n):
    alist = []
    for i in range(n):
        for j in range(i+1, n, 1):
            alist.append((i,j))
    return alist

In [101]:
pairlist = getpair(len(movielist))

In [102]:
len(pairlist)

47273226

In [103]:
def change_struc(alist):
    m = alist[0]
    u = []
    r = []
    for i in range(len(alist[1])):
        u.append(alist[1][i][0])
        r.append(alist[1][i][1])
    return([m,u,r])

In [104]:
movielist = sc.parallelize(movielist).map(change_struc).collect()

## Calculate the cosine similarity

In [105]:
def Rxyi(x, y):
    intersection = list(set(x[1]).intersection(set(y[1])))
    rxy_list = []
    for i in range(len(intersection)):
        rxy_list.append(x[2][i]*y[2][i])
    return(sum(rxy_list))

In [16]:
def sqrt(x):
    return x**2

In [106]:
def sim(x,y):
    sim = 0
    intersection = set(x[1]).intersection(set(y[1]))
    if len(list(intersection))!= 0:
        Rxy = Rxyi(x,y)
        Rx = sum(map(sqrt, x[2]))**0.5
        Ry = sum(map(sqrt, y[2]))**0.5
        sim = Rxy/(Rx*Ry)
    return (x[0], y[0], sim)

In [107]:
sim_list = []
for i in range(movie_num):
    item = movielist.pop(0)
    rddmovie = sc.parallelize(movielist)
    sim_list.extend(rddmovie.map(lambda j: sim(j, item)).collect())
    print(item[0])

53318
190
56022
5460
142074
133365
158882
8743
60737
27704
59026
2732
2467
5889
145
3726
110
7326
1281
60487
138966
25773
90245
99106
2797
111
60894
2607
473
8502
27317
100810
32914
5116
131023
499
1059
5602
8239
58162
3265
49822
4715
520
3581
4197
987
3635
78499
4546
1129
112852
47202
4238
6856
80590
5023
164226
1202
4773
3807
6202
12
4917
128089
2907
7055
89864
128991
3572
348
127194
34018
95796
91935
8264
4207
4215
2188
163072
7104
142115
190209
179
3633
6339
3619
2779
94130
8774
5113
26258
43871
5433
175585
180
184791
6598
5339
26700
3296
278
2080
6659
7093
72714
3287
71160
27772
949
102819
87306
6909
26694
41712
6820
69406
7781
48412
7982
90943
6954
130083
4847
59995
3431
2137
81834
32392
139642
3466
5773
4508
173197
53140
5573
1604
7132
78349
982
98122
105755
73017
4158
3316
70862
6022
258
1180
419
92938
44633
5975
3454
1937
122892
6383
7028
1826
47774
3450
30812
4777
88267
442
108188
49932
119167
3661
1529
4581
3578
6192
80880
6882
5650
65585
160646
150696
57526
74791
93006
8327

In [109]:
len(sim_list)

47273226

####  movie[i][1][u][0]:

#### i:movie

#### u:user

In [14]:
def getuser(x):
    return x[0]

In [15]:
def getuserrate(x):
    return x[1]

In [32]:
def rxyi(i):
    return rx[ux.index(i)]*ry[uy.index(i)]

In [19]:
def cossim(x,y):
    rx = sc.parallelize(x).map(getuserrate).collect()
    ry = sc.parallelize(y).map(getuserrate).collect()
    ux = sc.parallelize(x).map(getuser).collect()
    uy = sc.parallelize(y).map(getuser).collect()
    intersection = set(ux).intersection(set(uy))
    sim = 0
    if len(list(intersection))!= 0:
        Rxy = sum(sc.parallelize(list(intersection)).map(rxyi).collect())
        Rx = sum(sc.parallelize(rx).map(sqrt).collect())**0.5
        Ry = sum(sc.parallelize(ry).map(sqrt).collect())**0.5
        sim = Rxy/(Rx*Ry)
    return sim

In [ ]:
for i in range(len(pairlist)):
    #print(pairlist[i])
    a = pairlist[i][0]
    b = pairlist[i][1]
    
    rx = sc.parallelize(movielist[a][1]).map(getuserrate).collect()
    ry = sc.parallelize(movielist[b][1]).map(getuserrate).collect()
    ux = sc.parallelize(movielist[a][1]).map(getuser).collect()
    uy = sc.parallelize(movielist[b][1]).map(getuser).collect()
    intersection = set(ux).intersection(set(uy))
    sim = 0
    if len(list(intersection))!= 0:
        Rxy = sum(sc.parallelize(list(intersection)).map(rxyi).collect())
        Rx = sum(sc.parallelize(rx).map(sqrt).collect())**0.5
        Ry = sum(sc.parallelize(ry).map(sqrt).collect())**0.5
        sim = Rxy/(Rx*Ry)
    print('('+str(movielist[a][0])+','+str(movielist[b][0])+','+str(sim)+')')
    #print(sim)

In [ ]:
def pair_cossim(p):
    return cossim(movielist[p[0]][1], movielist[p[1]][1])

In [ ]:
sc.parallelize(pairlist).map(pair_cossim).collect()